# Step 2 NE204 Lab 1
## Optimization Routine Pt1: Raw Pulses -> Trapezoid Heights
__Dates: 08/2022 to 10/2022.__

__Group: Megan Schiferl, Chris Lamb, Curtis Berger, Jisu Park__

__Contents:__
This notebook takes in raw pulses and returns the heights of signal trapezoids for 25 different combinations of gap and peaking time. This data will be used in Optimization Routine pt 2 for finding the ideal combination of gap and peaking time. This optimization routine finds trapezoid heights by fitting the signal trapezoid with a trapezoid.

    Section 1: Imports and Directory
    Section 2: Managable Chunks of Data
    Section 3: Functions
    Section 4: Optimization Routine
    
__Notes on Running:__ First of all - don't! I've uploaded the optimization routine results from this notebook to github, so just move on to the next step to use them. If you still want to try this out, as always make sure the directory and data file are correct, otherwise you can run it right through and you'll end up with 25 csv files containing trapezoid heights. It takes ~90 mins on my computer, but maybe yours is faster.

# Section 1: Imports and Directory

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
import pandas as pd
from scipy import signal
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
import os
from tqdm import notebook

In [ ]:
# change directory
os.chdir(r'C:/Users/megas/Documents/Cal/NEFall2022/Detectors204/')

#check current working directory
retval = os.getcwd()
print ("Current working directory %s" %retval)

In [ ]:
#Input the data file and find the keys
path = r"C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1\Test-928\Cs137-24in-2min.h5"
f = h5py.File(path, 'r')

length = input("How long was data taken?")
isotope = input("Which isotope was measured?")

# Section 2: Managable Chunks of Data
 - Since my computer has proven only able to handle ~5000 pulses at once, this is where I lump that data into groups of 5000 or less pulses for processing.

In [ ]:
#Save the data into an array for analysis
pulses = np.array(f['raw_data'])

f.close()
#Remove the last pulse due to timestamp information
pulses = pulses[:-140]

print("The number of pulses in this file is:", len(pulses))
print("The length of each pulse is:", len(pulses[0]))
print("The number of pulsesX array that you need is:", int(len(pulses)/5000) + 1)

In [ ]:
#Split the large files up into different pulsesX arrays
pulses1 = []
pulses2 = []
pulses3 = []
pulses4 = []
for i in notebook.tqdm(range(len(pulses))):
    if i < 5000:
        pulses1.append(pulses[i])
    elif i < 10000:
        pulses2.append(pulses[i])
    elif i < 15000:
        pulses3.append(pulses[i])
    else:
        pulses4.append(pulses[i])

pulse_names = [pulses1, pulses2, pulses3, pulses4]
        
print(len(pulses1), len(pulses2), len(pulses3), len(pulses4))

# Section 3: Functions
 - Defining the following functions: (credit to Chris for the code in these functions)
     - exp_func: the paramters for an exponential function
     - determine_rise: find the index of the start of the rise
     - delay_signal: delays the signal pulse by adding the indicated length of noise 
     - dkl: returns the dkl function used for trapezoidal filter 
     - sfunc: applies the trapezoidal filter (using dkl and applying pole-zero correction) 
     - fit_trap: a piecewise trapezoid for fitting a trapezoid to the signal trapezoids

In [ ]:
#defining an exponential function
def exp_func(x, a, b, c):
    #returns a times e^(-b times a) + c
    return a * np.exp(-b * x) + c
    
#Finding the start of rise index using V4 from the notes notebook
def determine_rise(signal, sigma=8, window=20, offset=15):
    noise_samp = signal[:500]
    mean, std = np.mean(noise_samp), np.std(noise_samp)

    grad = np.gradient(np.where(signal > mean+sigma*std, signal, 0))

    grad_pos, grad_neg = np.argwhere(grad>2), np.argwhere(grad<-2)

    rise_start = 0
    for gp in grad_pos:
        close = False
        for gn in grad_neg:
            if (gp-gn < window and gp-gn >0):
                close = True
        if not close:
            rise_start = gp
            break

    return int(rise_start-offset)

#Define the trapezoidal filter and signal output function
def delay_signal(signal, delay, p=500):
    np.random.seed(9)
    noise_samp = signal[:p]
    mean, std = np.mean(noise_samp), np.std(noise_samp)
    noise = np.random.normal(mean, 0.8*std, delay)
    return np.hstack([noise, signal])

def dkl(signal, i, k, l, w=0):
    if w == 0:
        w = int(2.5*k+l)
    vj = signal[i+w:i+w+w] 
    vjk = signal[i+w-k:i+w-k+w]
    vjl = signal[i+w-l:i+w-l+w]
    vjkl = signal[i+w-k-l:i+w-k-l+w]
    dkl_s = vj - vjk - vjl + vjkl
    return dkl_s

def sfunc(signal, start_rise, tau, peaking_time, gap_time, w=0):
    if w == 0:
        w = int(2.5*peaking_time+gap_time)
    ss = []
    dkl_s = dkl(signal, start_rise, peaking_time, peaking_time+gap_time, w)
    for j in range(w):
        if j == 0:
            ss.append(0)
        else:
            ss.append(ss[j-1]*(1+1/tau)+ dkl_s[j])
    return np.array(ss)

def fit_trap(x, topleft, topright, top, ps, pe, e):
    #Trapezoid signal shape
    tr = np.zeros(len(x))
    tr[:int(ps)] = 0
    tr[int(ps):int(topleft)] = ((top-0)/(topleft-ps))*(x[int(ps):int(topleft)]-ps)+0
    tr[int(topleft):int(topright)] = top
    tr[int(topright):int(pe)] = ((e-top)/(pe-topright))*(x[int(topright):int(pe)]-pe)+e
    tr[int(pe):] = e
    return tr

# Section 4: Optimization Algorithm 
 - For different m, k combinations, the raw pulses are fitted into trapezoids, then fits a trapezoid to the signal trapezoid, saving the heights of the traps

In [ ]:
allfs = []
startrises = []
for i in notebook.tqdm(range(len(pulse_names)), desc = 'Iterating through pulse chunks', leave=False):
    pulsesX = pulse_names[i]

    #remove saturated signals
    sat = np.amax(np.amax(pulsesX))

    nonsat_pulses = []
    for a in notebook.tqdm(range(len(pulsesX)), desc = "Removing Saturated Pulses", leave = False):
        max_y = np.amax(pulsesX[a])
        if (max_y != sat):
            nonsat_pulses.append(pulsesX[a])

    nonsat_pulses = np.array(nonsat_pulses)  

    #Background subtraction for the raw data
    n = len(nonsat_pulses)
    pulses_sub = []
    for c in notebook.tqdm(range(n), desc = "Noise Floor Subtraction", leave = False):
        bkg = np.mean(nonsat_pulses[c][0:500])
        pulses_sub.append(nonsat_pulses[c] - bkg)

    pulses_sub = np.array(pulses_sub)

    #filtering the signals
    fs = savgol_filter(pulses_sub, 53, 2)

    #Place all i in an array
    startrise = []
    for q in notebook.tqdm(range(len(fs)), desc = "Finding start-rise values", leave = False):
        itemp = int(determine_rise(fs[q]))
        startrise.append(itemp)
    
    #Save them all for use in the next chunk of analysis
    startrises.append(startrise)
    allfs.append(fs)

In [ ]:
for n in notebook.tqdm(range(5), desc = "Iterating through Gap Times:"):
    m = 200 + n*200
    
    for b in notebook.tqdm(range(5), desc  = "Iterating through Peaking Times:"):
        k = b*200 + 150
        l = k+m
        Tau = 13450 #int(tau_avg)
        allheights = []   

        for i in notebook.tqdm(range(len(allfs)), desc = 'Iterating through all signals', leave=False):
            
            #Delay the signals 
            dfs = []
            d=int(2.5*k+m)
            traps = []
            
            for r in notebook.tqdm(range(len(allfs[i])), leave = False):
                tempfs = delay_signal(allfs[i][r],delay=d)
                dfs.append(tempfs)

            #Calculate the trapezoids
            for t in notebook.tqdm(range(len(dfs)), leave = False):
                trap = sfunc(dfs[t], startrises[i][t], Tau, k, m)
                traps.append(trap)
            
            traps = np.array(traps)
            
            #Trapezoid Fitting Method
            #fitting the signal trapezoids using fit_trap
            h1 = []
            for v1 in notebook.tqdm(range(len(traps)), leave = False):
                try:
                    popt, pcov = curve_fit(fit_trap, np.arange(len(traps[v1])), traps[v1], p0=[400, 1200, 100000, 100, 1400, 1500])
                    h1.append(popt)
                except:
                    h1.append(np.zeros(6))

            #Trapezoid Fit method cont.
            h2 = []
            #Remove the zero arrays from h1
            for a in range(len(h1)):
                result = h1[a].all(0)
                if result == True:
                    h2.append(h1[a])
            h2 = np.array(h2)

            #find the mean of the top of traps
            height2 = []
            for b in notebook.tqdm(range(len(h2)), leave = False):
                i1 = int(h2[b][3])
                i2 = int(h2[b][4])
                temph = np.mean(traps[b][i1:i2+1])
                height2.append(temph)
            height = list(filter(lambda a: a >= 0, height2))
            allheights = allheights + height
            
        allheights = np.array(allheights)
        
        # change directory
        os.chdir(r'C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1\TrapHeights2')

        #Let's just save these heights to a file
        name = "{}_{}_k{}m{}.csv".format(isotope,length,k,m)
        np.savetxt(name, allheights, delimiter=",")
        
        del allheights